In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession.builder
         .master("local[*]")
         .appName("Taxi Trips To Parquet")
         .getOrCreate()
        )

22/02/26 08:57:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [4]:
df_green = (df_green
             .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime')
             .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')
)

df_yellow = (df_yellow
             .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')
             .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')
)

In [5]:
common_columns = [col for col in df_green.columns if col in set(df_yellow.columns)]

In [7]:
from pyspark.sql import functions as F

df_green_sel = (df_green
                .select(common_columns)
                .withColumn('service_type', F.lit('green'))
               )

df_yellow_sel = (df_yellow
                .select(common_columns)
                .withColumn('service_type', F.lit('yellow'))
               )

In [8]:
df_trip_data = df_green_sel.unionAll(df_yellow_sel)

In [11]:
df_trip_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [14]:
df_trip_data.raegisterTempTable('trips_data')

In [15]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [16]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [17]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')